### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [20]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [21]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [22]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [23]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [24]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [25]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['house']

47438

In [26]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [27]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [28]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [29]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 2800
print(newsgroups_train.data[idx])

Archive-name: space/net
Last-modified: $Date: 93/04/01 14:39:15 $

NETWORK RESOURCES

OVERVIEW

    You may be reading this document on any one of an amazing variety of
    computers, so much of the material below may not apply to you. In
    general, however, systems connected to 'the net' fall in one of three
    categories: Internet, Usenet, or BITNET. Electronic mail may be sent
    between these networks, and other resources available on one of these
    networks are sometimes accessible from other networks by email sent to
    special 'servers'.

    The space and astronomy discussion groups actually are composed of
    several mechanisms with (mostly) transparent connections between them.

    One mechanism is the mailing list, in which mail is sent to a central
    distribution point which relays it to all recipients of the list. In
    addition to the general lists for space (called SPACE Digest for
    Internet users, and SPACE on BITNET), there are a number of more
    speci

In [30]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [31]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.54488082, 0.49799339, ..., 0.        , 0.        ,
       0.        ])

In [32]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 2800,  5729,  9096, ...,  5802, 10187,  9314], dtype=int64)

In [33]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [34]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'sci.space'

In [35]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

sci.space
sci.space
sci.space
sci.space
sci.space


### Modelo de clasificación Naïve Bayes

In [36]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [37]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [38]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Parte 1

Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [39]:
# Lista de documentos
idx_list = [855, 4224, 6045, 1274, 2124]

#### Documento 1

In [40]:
idx = idx_list[0]

# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

# podemos ver los valores de similaridad ordenados de mayor a menos
sim = np.sort(cossim)[::-1]
print("---------------------------------")
print("Similaridades: ")
print(sim)

# y a qué documentos corresponden
sim_docs = np.argsort(cossim)[::-1]
print("---------------------------------")
print("Documentos similares: ")
print(sim_docs)

# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

# el documento original pertenece a la clase:
main_class = newsgroups_train.target_names[y_train[idx]]
print("---------------------------------")
print("Clase del documento original: ")
print(main_class)

# y los 5 más similares son de las clases:
print("---------------------------------")
print("Clase de los documentos similares: ")
for i in mostsim:
    print(newsgroups_train.target_names[y_train[i]])

---------------------------------
Similaridades: 
[1.         0.42192677 0.31493766 ... 0.         0.         0.        ]
---------------------------------
Documentos similares: 
[  855 10562   225 ...   473  9095  4021]
---------------------------------
Clase del documento original: 
talk.politics.guns
---------------------------------
Clase de los documentos similares: 
talk.politics.guns
talk.politics.guns
talk.politics.guns
talk.politics.guns
talk.politics.guns


In [41]:
# Vemos el documento original
print("---------------------------------")
print('Documento original:')
print("---------------------------------")
print(newsgroups_train.data[idx])

# Leemos el documento más similar (después del original) a modo de verificación
print("")
print("---------------------------------")
print("Primer documento más similar: ")
print("---------------------------------")
print(newsgroups_train.data[mostsim[1]])

---------------------------------
Documento original:
---------------------------------

The point that I forgot to bring up here (and this has nothing to do with being
a gang member or not) is that it is illegal to carry a concealed weapon in this
area (or in the state of illinois for that matter).  This is not to say that
people in Illinois don't carry concealed weapons illegaly but practicing like
that when there are other people around wasn't too bright of an idea.


I agree.      If you don't practice at all and carry a gun for self-defense you
most likely would be in big trouble if a situation were to arise.

---------------------------------
Primer documento más similar: 
---------------------------------
A couple of questions for you firearms law experts out there:  

Question #1

According to the NRA/ILA state firearms lawbook, in Wisconsin it is
'unlawful for any person except a peace officer to go armed* with a 
"concealed and dangerous weapon."  There is no statutory provis

#### Documento 2

In [42]:
idx = idx_list[1]

# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

# podemos ver los valores de similaridad ordenados de mayor a menos
sim = np.sort(cossim)[::-1]
print("---------------------------------")
print("Similaridades: ")
print(sim)

# y a qué documentos corresponden
sim_docs = np.argsort(cossim)[::-1]
print("---------------------------------")
print("Documentos similares: ")
print(sim_docs)

# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

# el documento original pertenece a la clase:
main_class = newsgroups_train.target_names[y_train[idx]]
print("---------------------------------")
print("Clase del documento original: ")
print(main_class)

# y los 5 más similares son de las clases:
print("---------------------------------")
print("Clase de los documentos similares: ")
for i in mostsim:
    print(newsgroups_train.target_names[y_train[i]])

---------------------------------
Similaridades: 
[1.         0.33213845 0.32931617 ... 0.         0.         0.        ]
---------------------------------
Documentos similares: 
[ 4224  8211  2328 ... 10266  7683  3782]
---------------------------------
Clase del documento original: 
talk.politics.misc
---------------------------------
Clase de los documentos similares: 
talk.politics.misc
alt.atheism
talk.politics.misc
talk.politics.misc
sci.crypt


In [43]:
# Vemos el documento original
print("---------------------------------")
print('Documento original:')
print("---------------------------------")
print(newsgroups_train.data[idx])

# Leemos el documento más similar (después del original) a modo de verificación
print("")
print("---------------------------------")
print("Primer documento más similar: ")
print("---------------------------------")
print(newsgroups_train.data[mostsim[1]])

---------------------------------
Documento original:
---------------------------------
# #They believe that they have a right to FORCE people to hire them,
# #rent to them, and do business with them, regardless of the feelings
# #or beliefs of the other person.
# 
# Cramer, you are off your target again.  The law *forces* no one to obey
# it.  At every point any individual may stand up and say *this law
# sucks*.  Even you could say this.  Gay men and women have not *forced*

You mean they passed a law that does nothing at all?  No enforcement
mechanisms?  As usual, you are wrong.

# any off this.  Changes in the law have been brought about by
# democratic* processes, those same processes are the ones that protect
# you from certain abuses.

Yeah, right.  I guess the next time a homosexual complains about
sodomy laws, I can just echo your stupidity about "democratic
processes" and he won't have any basis for complaint.

# #I must admit that I never understood why it is referred to as 

#### Documento 3

In [44]:
idx = idx_list[2]

# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

# podemos ver los valores de similaridad ordenados de mayor a menos
sim = np.sort(cossim)[::-1]
print("---------------------------------")
print("Similaridades: ")
print(sim)

# y a qué documentos corresponden
sim_docs = np.argsort(cossim)[::-1]
print("---------------------------------")
print("Documentos similares: ")
print(sim_docs)

# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

# el documento original pertenece a la clase:
main_class = newsgroups_train.target_names[y_train[idx]]
print("---------------------------------")
print("Clase del documento original: ")
print(main_class)

# y los 5 más similares son de las clases:
print("---------------------------------")
print("Clase de los documentos similares: ")
for i in mostsim:
    print(newsgroups_train.target_names[y_train[i]])

---------------------------------
Similaridades: 
[1.         0.57423947 0.26629149 ... 0.         0.         0.        ]
---------------------------------
Documentos similares: 
[6045 6004 8107 ... 3939 9610 4386]
---------------------------------
Clase del documento original: 
comp.os.ms-windows.misc
---------------------------------
Clase de los documentos similares: 
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.os.ms-windows.misc
comp.sys.mac.hardware
comp.graphics


In [45]:
# Vemos el documento original
print("---------------------------------")
print('Documento original:')
print("---------------------------------")
print(newsgroups_train.data[idx])

# Leemos el documento más similar (después del original) a modo de verificación
print("")
print("---------------------------------")
print("Primer documento más similar: ")
print("---------------------------------")
print(newsgroups_train.data[mostsim[1]])

---------------------------------
Documento original:
---------------------------------


    >Version 1.3 drivers are due to be release by Cirrus soon.
    >Unfortunately, their not available via FTP, you have to dial
    >up their BBS in the USA.  I do this from NZ using a 14.4k modem
    >to cut down on phone bills.  It took me around 7 minutes to 
    >download the v1.2 driver.

	Could you please upload to any of the ftp sites (such as
	ftp.ciaca.indiana.edu) and announce it here? This will benefit
	people does not have access to their BBS in USA (like me :-))?

	Thanks a lot.

---------------------------------
Primer documento más similar: 
---------------------------------
Could anybody please provide me a copy of the Windows 3.1 drivers and grabbers
from Orchid Technologies for use with their ProDesigner IIs ISA video card? Currently I do not have access to a modem to dial out to Orchid BBS.
If you can help me, please do any of the following, wichever is most convenient
to you:


#### Documento 4

In [46]:
idx = idx_list[3]

# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

# podemos ver los valores de similaridad ordenados de mayor a menos
sim = np.sort(cossim)[::-1]
print("---------------------------------")
print("Similaridades: ")
print(sim)

# y a qué documentos corresponden
sim_docs = np.argsort(cossim)[::-1]
print("---------------------------------")
print("Documentos similares: ")
print(sim_docs)

# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

# el documento original pertenece a la clase:
main_class = newsgroups_train.target_names[y_train[idx]]
print("---------------------------------")
print("Clase del documento original: ")
print(main_class)

# y los 5 más similares son de las clases:
print("---------------------------------")
print("Clase de los documentos similares: ")
for i in mostsim:
    print(newsgroups_train.target_names[y_train[i]])

---------------------------------
Similaridades: 
[1.         0.55560698 0.39695861 ... 0.         0.         0.        ]
---------------------------------
Documentos similares: 
[ 1274  5872   367 ...   340 10639 10545]
---------------------------------
Clase del documento original: 
comp.windows.x
---------------------------------
Clase de los documentos similares: 
comp.windows.x
comp.windows.x
comp.windows.x
comp.windows.x
comp.windows.x


In [47]:
# Vemos el documento original
print("---------------------------------")
print('Documento original:')
print("---------------------------------")
print(newsgroups_train.data[idx])

# Leemos el documento más similar (después del original) a modo de verificación
print("")
print("---------------------------------")
print("Primer documento más similar: ")
print("---------------------------------")
print(newsgroups_train.data[mostsim[1]])

---------------------------------
Documento original:
---------------------------------
Hello Motif World,

a few days ago I posted my announcement for an update of Motif++. I got
several requests to send the bindings per e-mail, and I know of several people
who have been using Motif++, and there are probably a number of people I am
not aware of who are also using Motif++.

My question is:

How many people 'out there' would be interested to join a mailing-list, where
people can ask questions about Motif++, swap stories, and give new ideas about
new directions and improvements for the bindings. This would benefit the
user-community, as well as give me more insight in what people would like to
see added to Motif++. Motif++ is still very much a voluntary project, and this
way I can make a list of priorities, in what order things should be added, or
changed.

If you're interested in joining such a mailing-list, please take the time to
reply to this message, and tell me so. When there is su

#### Documento 5

In [48]:
idx = idx_list[4]

# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

# podemos ver los valores de similaridad ordenados de mayor a menos
sim = np.sort(cossim)[::-1]
print("---------------------------------")
print("Similaridades: ")
print(sim)

# y a qué documentos corresponden
sim_docs = np.argsort(cossim)[::-1]
print("---------------------------------")
print("Documentos similares: ")
print(sim_docs)

# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

# el documento original pertenece a la clase:
main_class = newsgroups_train.target_names[y_train[idx]]
print("---------------------------------")
print("Clase del documento original: ")
print(main_class)

# y los 5 más similares son de las clases:
print("---------------------------------")
print("Clase de los documentos similares: ")
for i in mostsim:
    print(newsgroups_train.target_names[y_train[i]])

---------------------------------
Similaridades: 
[1.         0.84995459 0.42025854 ... 0.         0.         0.        ]
---------------------------------
Documentos similares: 
[2124 4915 7130 ... 2424 3258 9917]
---------------------------------
Clase del documento original: 
sci.med
---------------------------------
Clase de los documentos similares: 
sci.med
sci.med
sci.med
sci.med
talk.politics.guns


In [49]:
# Vemos el documento original
print("---------------------------------")
print('Documento original:')
print("---------------------------------")
print(newsgroups_train.data[idx])

# Leemos el documento más similar (después del original) a modo de verificación
print("")
print("---------------------------------")
print("Primer documento más similar: ")
print("---------------------------------")
print(newsgroups_train.data[mostsim[1]])

---------------------------------
Documento original:
---------------------------------

Hismanal (astemizole) is most definitely linked to weight gain.
It really is peculiar that some antihistamines have this effect,
and even more so an antihistamine like astemizole which purportedly
doesn't cross the blood-brain barrier and so tends not to cause
drowsiness.


---------------------------------
Primer documento más similar: 
---------------------------------

So antihistamines can cause weight gain.  NOW they tell me. :-)
Is there any way to find out which do & which don't?  My doctor
obviously is asleep at the wheel.

The original poster mentioned fatigue.  I had that too, but it was
mostly due to the really bizarre dreams I was having -- I wasn't getting
any rest.  My doctor said that was a common reaction.  If astemizole
doesn't cross the blood-brain barrier, how does it cause that side
effect?  Any ideas?

-- 


### Parte 2

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [50]:
from sklearn.model_selection import GridSearchCV

#### MultinomialNB

Vectorización

In [51]:
# instanciamos un vectorizador
tfidfvect_multi = TfidfVectorizer(stop_words='english')

# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF) y transformar directamente los datos
X_train_multi = tfidfvect_multi.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

Modelo

In [52]:
# Se definen varios parametros del modelo
parametros = {
    'alpha': [0.01, 0.1, 0.5, 1.0], # Default es 1.0 -> Parámetro de suavizado de Laplace
    'fit_prior': [True, False] # Default es True -> Aprende las probabilidades a priori de las clases? Sino, usa probabilidad uniforme
}

In [53]:
clf_multi = GridSearchCV(MultinomialNB(), parametros, cv=5, scoring='f1_macro')
clf_multi.fit(X_train_multi, y_train)
best_params_multi = clf_multi.best_params_
clf_multi_best = clf_multi.best_estimator_

# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test_multi = tfidfvect_multi.transform(newsgroups_test.data)
y_test_multi = newsgroups_test.target

# Evaluamos el modelo con los mejores parametros
y_pred_multi = clf_multi_best.predict(X_test_multi)

# Obtenemos el F1-score con "macro" por ser mejor para datasets desbalanceados
f1_multi = f1_score(y_test_multi, y_pred_multi, average='macro')

print(f"Mejores parámetros con MultinomialNB: {best_params_multi}")
print(f"Mejor F1-score con MultinomialNB: {f1_multi}")

Mejores parámetros con MultinomialNB: {'alpha': 0.01, 'fit_prior': False}
Mejor F1-score con MultinomialNB: 0.6876839515570421


#### ComplementNB

Vectorización

In [54]:
# instanciamos un vectorizador
tfidfvect_cnb = TfidfVectorizer(stop_words='english')

# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF) y transformar directamente los datos
X_train_cnb= tfidfvect_cnb.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

Modelo

In [55]:
clf_cnb = GridSearchCV(ComplementNB(), parametros, cv=5, scoring='f1_macro')
clf_cnb.fit(X_train_cnb, y_train)
best_params_cnb = clf_cnb.best_params_
clf_cnb_best = clf_cnb.best_estimator_

# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test_cnb = tfidfvect_cnb.transform(newsgroups_test.data)
y_test_cnb = newsgroups_test.target

# Evaluamos el modelo con los mejores parametros
y_pred_cnb = clf_cnb_best.predict(X_test_cnb)

# Obtenemos el F1-score con "macro" por ser mejor para datasets desbalanceados
f1_cnb = f1_score(y_test_cnb, y_pred_cnb, average='macro')

print(f"Mejores parámetros con ComplementNB: {best_params_cnb}")
print(f"Mejor F1-score con ComplementNB: {f1_cnb}")

Mejores parámetros con ComplementNB: {'alpha': 0.1, 'fit_prior': True}
Mejor F1-score con ComplementNB: 0.6919194498508968


#### **Comentarios**

- Ya solo con agregar los "Stop Words" del inglés se mejora el score del modelo.
- La diferencia en el score no es tanta entre ambos modelos, pero ComplementNB es mejor en este caso.

### Parte 3
Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [102]:
# Quitamos las stopwords
tfidfvect_t = TfidfVectorizer(stop_words='english')
X_train_= tfidfvect_t.fit_transform(newsgroups_train.data)

# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect_t.vocabulary_.items()}


# Se transpone la matriz de entrenamiento
X_train_T = X_train_.T

In [129]:
# Se seleccionan las palabras y se obtiene su indice

palabras = ['image', 'god', 'house', 'space', 'car']
word_index = []

for palabra in palabras:
    idx = tfidfvect_t.vocabulary_[palabra]
    print(f"Indice de la palabra '{palabra}': {idx}")
    word_index.append(idx)

Indice de la palabra 'image': 48959
Indice de la palabra 'god': 43733
Indice de la palabra 'house': 47314
Indice de la palabra 'space': 83871
Indice de la palabra 'car': 25717


In [130]:
for palabra_idx in word_index:
    # midamos la similaridad coseno con todos los términos de train
    terms_cossim = cosine_similarity(X_train_T[palabra_idx], X_train_T)[0]
    print("---------------------------------")
    print(f"Palabra: {idx2word[palabra_idx]}")
    print("---------------------------------")
    # los 5 términos más similares:
    terms_mostsim_idx = np.argsort(terms_cossim)[::-1][1:6]
    words_mostsim = [idx2word[i] for i in terms_mostsim_idx]
    terms_mostsim_cos = np.sort(terms_cossim)[::-1][1:6]
    print(f"Palabras similares: {words_mostsim}")
    print(f"Similitud: {terms_mostsim_cos}")
    print("")
    

---------------------------------
Palabra: image
---------------------------------
Palabras similares: ['morphological', 'vista', 'ffts', 'histogram', 'nimh']
Similitud: [0.27452917 0.27262271 0.27262271 0.26931519 0.26091651]

---------------------------------
Palabra: god
---------------------------------
Palabras similares: ['jesus', 'bible', 'christ', 'faith', 'existence']
Similitud: [0.28062629 0.27639079 0.26683025 0.25928239 0.25887389]

---------------------------------
Palabra: house
---------------------------------
Palabras similares: ['senate', 'white', 'cpr', 'veto', 'miyazawa']
Similitud: [0.26694565 0.25493653 0.24317101 0.21345921 0.20213188]

---------------------------------
Palabra: space
---------------------------------
Palabras similares: ['nasa', 'shuttle', 'seds', 'enfant', 'exploration']
Similitud: [0.32793997 0.2902489  0.28490722 0.26941108 0.23983273]

---------------------------------
Palabra: car
---------------------------------
Palabras similares: ['cars

#### Estudio de las palabras y sus similares:

**image**:
- ``morphological``: tiene sentido.
- ``vista``: puede relacionarse a una imagen.
- ``ffts``: puede tratarse de Fast Fourier Transforms (FTTs) que se asocian a imágenes.
- ``nimh``: no parece tener sentido en este caso, podría ser una palabra no completa.

**god**:
- En este caso, claramente todas las palabras ``['jesus', 'bible', 'christ', 'faith', 'existence']`` tienen sentido y relación con la palabra "Dios".

**house**:
- ``['senate', 'white', 'veto']`` son todas palabras que pueden relacionarse con política estadounidense, cuyas instituciones utilizan la palabra "Hause" para sus nombres.
- ``['cpr', 'miyazawa']``: estas palabras no parecen tener mucho sentido en su relación con "House".

**space**
- ``['nasa', 'shuttle', 'exploration']``: todas estas palabras pueden relacionarse fácilmente con la palabra "space".
- ``seds``: esta palabra puede relacionarse a una organización estudiantil llamada SEDS que promueve la exploración espacial.
- ``enfant``: esta palabra parece fuera de lugar en este conjunto.

**car**
- ``['cars', 'dealer', 'civic', 'owner']``: todas estas palabras pueden asociarse a carros (en este caso 'civi' se refiere al Honda Civic).
- ``criterium``: Esta palabra no parece tener mucho sentido en este conjunto.

#### Potencial mejora

- Buscar los 5 documentos más relevantes para las palabras que no parecen tener relación y encontrar sus categorías. Asociar la categoría a la cantidad de ocurrencias de la palabra en dicha categoría. De este modo se puede tratar de encontrar una exlicación a la relación entre las palabras.
- Leer algunos documentos que contengan la palabra original, la palabra que no parece encajar en la categoría y alguna palabra que sí parece correcta. Esto puede ayudar a entender la relación entre ellas.